# LTCM Risk Decomposition

In [9]:

import pandas as pd
import numpy as np
from pathlib import Path
import statsmodels.api as sm

PERIODS_PER_YEAR = 12

def load_ltcm():
    df = pd.read_excel(
        r'C:\Users\Zara\Documents\GitHub\prm-group-c\data\ltcm_exhibits_data.xlsx',
        sheet_name='Exhibit 2',
        skiprows=4,
        names=['date','fund_capital','gross_return','net_return','net_index']
    )
    df = df[pd.to_datetime(df['date'], errors='coerce').notna()].copy()
    df['date'] = pd.to_datetime(df['date'])
    df['month'] = df['date'].dt.to_period('M')
    df = df.set_index('month')[['gross_return','net_return']].astype(float)
    return df

def load_spy():
    spy = pd.read_excel(r'C:\Users\Zara\Documents\GitHub\prm-group-c\data\spy_data.xlsx', sheet_name='excess returns')
    spy['date'] = pd.to_datetime(spy['date'])
    spy['month'] = spy['date'].dt.to_period('M')
    return spy.set_index('month')[['SPY']].rename(columns={'SPY':'spy_excess'})

def annualized_stats(series):
    ann_mean = series.mean() * PERIODS_PER_YEAR
    ann_vol = series.std(ddof=1) * np.sqrt(PERIODS_PER_YEAR)
    sharpe = ann_mean / ann_vol
    skew = series.skew()
    kurt = series.kurt()
    q5 = series.quantile(0.05)
    return ann_mean, ann_vol, sharpe, skew, kurt, q5

ltcm = load_ltcm()
spy = load_spy()
combined = ltcm.join(spy, how='inner').dropna()
combined.head()


,gross_return,net_return,spy_excess
month,,,
1994-04,0.014,0.008,0.007996
1994-05,0.068,0.053,0.012464
1994-06,-0.039,-0.029,-0.032782
1994-07,0.116,0.084,0.028768
1994-08,0.038,0.030,0.034321


Q2.1 Summary statistics (annualized + tails)

In [10]:

stats = {}
for col in ['gross_return','net_return','spy_excess']:
    stats[col] = annualized_stats(combined[col])
summary = pd.DataFrame(stats, index=[
    'Annualized mean','Annualized vol','Sharpe ratio','Skewness','Excess kurtosis','5th percentile'
])
summary.round(3)


,gross_return,net_return,spy_excess
Annualized mean,0.302,0.214,0.169
Annualized vol,0.137,0.112,0.111
Sharpe ratio,2.212,1.912,1.526
Skewness,-0.340,-0.871,-0.413
Excess kurtosis,1.646,3.058,-0.272
5th percentile,-0.027,-0.023,-0.047



Gross excess returns compound at roughly **30.2%** per year with a 13.7% volatility and a Sharpe ratio of **2.21**.
Net returns (after fees) drop to **21.4%** with 11.2% volatility and a Sharpe of **1.91**.
SPY's excess return over the same window is **17.0%** with 11.1% volatility and a Sharpe of **1.53**.
Both LTCM series have noticeably fatter left tails than SPY: net return skewness is **?0.87** and kurtosis **3.06**, with a
5th percentile of **?2.3%** compared with SPY's **?4.7%**. The fee drag therefore erodes roughly **8.8 percentage points** of annualized performance.


Q2.2 Compare to SPY and gross vs net


Relative to SPY, LTCM delivered equity-like volatility but dramatically higher mean excess returns and fatter tails.
The gross/net spread makes it clear that fees reduce both alpha and Sharpe materially?net Sharpe falls about 30 bps.
SPY's skew and kurtosis remain milder, and its 5% drawdown is worse than LTCM's, highlighting how levered relative-value
trades can be less synchronized with broad equities until stress events hit.


Q2.3 Linear factor decomposition (SPY)

In [11]:

y = combined['net_return']
X = sm.add_constant(combined['spy_excess'])
lin_model = sm.OLS(y, X).fit()
lin_alpha = lin_model.params['const'] * PERIODS_PER_YEAR
lin_beta = lin_model.params['spy_excess']
lin_r2 = lin_model.rsquared
lin_alpha, lin_beta, lin_r2


(0.19436315229381984, 0.1168585140531083, 0.013396747524952235)


The SPY-only model delivers an annualized alpha of **19.4%**, beta **0.12**, and R? ? **1.3%**. The small beta confirms
that the strategy's performance is largely orthogonal to the market. The positive alpha is still economically large after fees,
so LTCM generated substantial excess performance beyond what SPY could explain.


Q2.4 Non-linear exposure (quadratic SPY)

In [12]:

combined['spy_sq'] = combined['spy_excess'] ** 2
quad_model = sm.OLS(y, sm.add_constant(combined[['spy_excess','spy_sq']])).fit()
quad_alpha = quad_model.params['const'] * PERIODS_PER_YEAR
quad_beta = quad_model.params['spy_excess']
quad_gamma = quad_model.params['spy_sq']
quad_r2 = quad_model.rsquared
quad_alpha, quad_beta, quad_gamma, quad_r2


(0.21439260270551067,
 0.14309362466645265,
 -1.6920013002872807,
 0.017578469177554124)


Adding a quadratic market term increases R? only marginally to **1.76%**. The concave quadratic coefficient (?1.69)
indicates that LTCM's payoff profile bends downward in large market moves?akin to being short convexity. The annualized alpha
remains around **21%**, reinforcing that most performance is unrelated to simple market direction.


Q2.5?Q2.7 Threshold up/down exposure

In [13]:

k1, k2 = 0.03, -0.03
mkt = combined['spy_excess']
X_thr = pd.DataFrame({
    'market': mkt,
    'up': (mkt - k1).clip(lower=0),
    'down': (k2 - mkt).clip(lower=0)
})
thr_model = sm.OLS(y, sm.add_constant(X_thr)).fit()
thr_alpha = thr_model.params['const'] * PERIODS_PER_YEAR
thr_params = thr_model.params.drop('const')
thr_r2 = thr_model.rsquared
thr_alpha, thr_params.to_dict(), thr_r2


(0.1585248508794918,
 {'market': 0.44259447147716713,
  'up': -0.7342558697967847,
  'down': 1.349338945922227},
 0.047871598779570035)


The threshold regression yields:
- Annualized alpha ~ **15.8%**.
- Market beta ~ **0.44** when returns are within 3%.
- Up-market slope of **0.73**, implying the strategy underperforms during very strong rallies (behaves like it is short calls).
- Down-market slope of **+1.35**, so losses accelerate when SPY falls beyond ?3%, mimicking short put exposure.
- R? rises to **4.8%**, so the non-linear terms explain more variation than the pure linear fit, but most performance remains idiosyncratic.

The combination of negative convexity and stronger downside loading indicates LTCM was effectively **short market volatility**
its exposure pattern resembles being short crash protection and marginally short upside participation. This is consistent with
long carry/relative value positions that earn steady small gains but are vulnerable to large market dislocations.
